经验证：
* 1、test的时间顺序并不是全部都是时序的，排行榜是按时序排列后取前0.78为public，后0.22为private，所以以后的工作重心将落到train的后44%
* 2、测试集中私有部分从2018-07-24 09:00:00开始，有9173472条数据，包含全部的building_id
* 3、测试集中公共部分与私有部分对应的数据从2017-07-24 09:00:00开始，有9174900条数据，包含全部的building_id
* 4、重点为训练集中2017-07-24 09:00:00后的数据建模，有9003109条数据，占训练集0.445
* 5、为防止测试集中不评分的那部分不知道算不算在0.22里面，也为防止一定的过拟合，可以考虑重点关注2017-07月后的训练数据，占0.51
* 6、验证EDA1后，发现0-104号建筑5月20日前的大量为0的数据是有误的，训练集不应该对这部分数据进行过拟合，删除这部分数据训练对提高public评分很有帮助，遗憾的是，private并不包含这类数据
* 7、猜想某些大范围为0的数据都同0-104号一样是有误的，重点验证7月后的该种数据，测试是否能提高public

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')
import gc


# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import matplotlib.patches as patches

from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
pd.set_option('max_columns', 100)


py.init_notebook_mode(connected=True)
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../input/ashrae-energy-prediction/train.csv
../input/ashrae-energy-prediction/sample_submission.csv
../input/ashrae-energy-prediction/building_metadata.csv
../input/ashrae-energy-prediction/weather_train.csv
../input/ashrae-energy-prediction/weather_test.csv
../input/ashrae-energy-prediction/ashrae-energy-prediction.zip
../input/ashrae-energy-prediction/test.csv


In [2]:
# 导入数据, 只有train 和 test
train = pd.read_csv('../input/ashrae-energy-prediction/train.csv')
test = pd.read_csv('../input/ashrae-energy-prediction/test.csv')

In [3]:
# 时间戳转换日期
train["timestamp"] = pd.to_datetime(train["timestamp"])
test["timestamp"] = pd.to_datetime(test["timestamp"])

In [4]:
train_exception = pd.read_pickle('../output/fork-of-ashrae-eda-exception-label2/train_exception.pkl')
train['exception'] = train_exception.exception.values
del train_exception
gc.collect()

31

In [5]:
test_exception = pd.read_pickle('../output/fork-of-ashrae-eda-exception-label2/test_exception.pkl')
test['exception'] = test_exception.exception.values
del test_exception
gc.collect()

20

In [6]:
gc.collect()

20

In [7]:
# 1413个建筑有0，498个建筑有1，324个建筑有2，145个建筑有3
# [0,1]有132个，[0,2]64，[0,3]21, [1,2]13
# [0,1,2]220, [0,1,3]111
# [0,1,2,3]13

In [8]:
gc.collect()

40

In [9]:
# (0,0)到(104,0)从5月中旬才开始有读数, 其中有些在开始有读数之前有一些读数，不知什么情况
# 猜测该地区出现了大面积的电表故障或者大面积的电力故障，其中有极少建筑幸存或用于检修测试，所以有少量读数，5月中旬后修好才恢复该地区正常用电或读表

In [10]:
train.exception.value_counts(dropna=False)

 0    17761403
-1     1924531
 1      365029
-2       85465
 2       79672
Name: exception, dtype: int64

In [11]:
test.exception.value_counts(dropna=False)

 0.0    35435620
-1.0     3920902
 NaN     1239910
 1.0      771806
-2.0      170502
 2.0      158860
Name: exception, dtype: int64

In [12]:
def exception_label(bid, meter, start, end):
    print(f'标记bid={bid}, meter={meter}异常')
    train.loc[(train.building_id == bid)  & (train.meter == meter) & (train.timestamp >= start) & (train.timestamp <= end), 'exception'] = 3
    test.loc[(test.building_id == bid)  & (test.meter == meter) & (test.timestamp >= start.replace('2016', '2017')) & (test.timestamp <= end.replace('2016', '2017')), 'exception'] = 3
    test.loc[(test.building_id == bid)  & (test.meter == meter) & (test.timestamp >= start.replace('2016', '2018')) & (test.timestamp <= end.replace('2016', '2018')), 'exception'] = 3
    gc.collect()

In [13]:
for bid in range(105):
    if bid not in [29, 53, 40, 45, 46]:
        exception_label(bid, 0, '2016-01-01 00:00:00', '2016-05-20 17:00:00')
        
exception_label(40, 0, '2016-01-01 00:00:00', '2016-06-03 10:00:00')
exception_label(45, 0, '2016-01-01 00:00:00', '2016-06-30 12:00:00')
exception_label(46, 0, '2016-03-01 15:00:00', '2016-05-20 17:00:00')

exception_label(177, 0, '2016-06-03 23:00:00', '2016-06-25 00:00:00')

exception_label(269, 0, '2016-06-03 23:00:00', '2016-06-25 00:00:00')

exception_label(376, 0, '2016-05-22 23:00:00', '2016-05-31 23:00:00')

exception_label(503, 0, '2016-01-21 18:00:00', '2016-01-29 09:00:00')

exception_label(537, 0, '2016-07-16 00:00:00', '2016-07-25 15:00:00')

exception_label(545, 0, '2016-01-17 01:00:00', '2016-02-10 00:00:00')

exception_label(681, 0, '2016-01-01 00:00:00', '2016-04-25 23:00:00')

exception_label(733, 0, '2016-05-29 01:00:00', '2016-06-20 23:00:00')

exception_label(738, 0, '2016-10-15 01:00:00', '2016-10-22 23:00:00')
exception_label(738, 0, '2016-10-24 01:00:00', '2016-10-30 23:00:00')
exception_label(738, 0, '2016-11-01 01:00:00', '2016-11-13 23:00:00')

exception_label(739, 0, '2016-09-21 01:00:00', '2016-10-02 23:00:00')

exception_label(848, 0, '2016-01-15 00:00:00', '2016-03-19 23:00:00')

exception_label(909, 0, '2016-01-01 00:00:00', '2016-02-02 11:00:00')

exception_label(929, 0, '2016-01-22 11:00:00', '2016-02-02 07:00:00')

exception_label(986, 0, '2016-01-01 00:00:00', '2016-02-17 07:00:00')

exception_label(997, 0, '2016-01-01 00:00:00', '2016-06-23 07:00:00')

# exception_label(1066, 0, '2016-02-13 20:00:00', '2016-02-29 11:00:00')
train.loc[(train.building_id == 1066)  & (train.meter == 0) & (train.timestamp >= '2016-02-13 20:00:00') & (train.timestamp <= '2016-02-29 11:00:00'), 'exception'] = 3
test.loc[(test.building_id == 1066)  & (test.meter == 0) & (test.timestamp >= '2017-02-13 20:00:00') & (test.timestamp <= '2017-02-28 23:00:00'), 'exception'] = 3
test.loc[(test.building_id == 1066)  & (test.meter == 0) & (test.timestamp >= '2018-02-13 20:00:00') & (test.timestamp <= '2018-02-28 23:00:00'), 'exception'] = 3

exception_label(1240, 0, '2016-04-20 23:00:00', '2016-05-02 11:00:00')

exception_label(1265, 0, '2016-05-05 06:00:00', '2016-05-26 14:00:00')

exception_label(1303, 0, '2016-01-26 01:00:00', '2016-03-21 17:00:00')


exception_label(278, 0, '2016-11-27 20:00:00', '2016-12-06 07:00:00')

exception_label(817, 0, '2016-07-18 00:00:00', '2016-08-17 23:00:00')

exception_label(868, 0, '2016-07-23 00:00:00', '2016-08-05 23:00:00')

标记bid=0, meter=0异常
标记bid=1, meter=0异常
标记bid=2, meter=0异常
标记bid=3, meter=0异常
标记bid=4, meter=0异常
标记bid=5, meter=0异常
标记bid=6, meter=0异常
标记bid=7, meter=0异常
标记bid=8, meter=0异常
标记bid=9, meter=0异常
标记bid=10, meter=0异常
标记bid=11, meter=0异常
标记bid=12, meter=0异常
标记bid=13, meter=0异常
标记bid=14, meter=0异常
标记bid=15, meter=0异常
标记bid=16, meter=0异常
标记bid=17, meter=0异常
标记bid=18, meter=0异常
标记bid=19, meter=0异常
标记bid=20, meter=0异常
标记bid=21, meter=0异常
标记bid=22, meter=0异常
标记bid=23, meter=0异常
标记bid=24, meter=0异常
标记bid=25, meter=0异常
标记bid=26, meter=0异常
标记bid=27, meter=0异常
标记bid=28, meter=0异常
标记bid=30, meter=0异常
标记bid=31, meter=0异常
标记bid=32, meter=0异常
标记bid=33, meter=0异常
标记bid=34, meter=0异常
标记bid=35, meter=0异常
标记bid=36, meter=0异常
标记bid=37, meter=0异常
标记bid=38, meter=0异常
标记bid=39, meter=0异常
标记bid=41, meter=0异常
标记bid=42, meter=0异常
标记bid=43, meter=0异常
标记bid=44, meter=0异常
标记bid=47, meter=0异常
标记bid=48, meter=0异常
标记bid=49, meter=0异常
标记bid=50, meter=0异常
标记bid=51, meter=0异常
标记bid=52, meter=0异常
标记bid=54, meter=0异常
标记bid=55, 

In [14]:
gc.collect()

20

In [15]:
%%time
for bid in train[(train.exception == 3) & (train.meter == 0)].building_id.unique():
    train.loc[(train.building_id == bid) & (train.meter == 0) & (train.exception == 0), 'exception'] = -3
    test.loc[(test.building_id == bid) & (test.meter == 0) & (test.exception != 3), 'exception'] = -3

CPU times: user 1min, sys: 45.9 s, total: 1min 46s
Wall time: 1min 46s


In [16]:
train.exception.value_counts(dropna=False)

 0    16686286
-1     1924531
-3      713041
 1      365029
 3      363028
-2       85465
 2       78720
Name: exception, dtype: int64

In [17]:
test.exception.value_counts(dropna=False)

 0.0    33291288
-1.0     3920902
-3.0     1463098
 NaN     1228508
 1.0      771806
 3.0      726902
-2.0      147834
 2.0      147262
Name: exception, dtype: int64

In [18]:
exception_label(43, 1, '2016-04-04 18:00:00', '2016-05-01 18:00:00')
exception_label(43, 1, '2016-05-01 20:00:00', '2016-06-06 11:00:00')

exception_label(60, 1, '2016-01-01 00:00:00', '2016-04-23 21:00:00')
exception_label(60, 1, '2016-04-23 23:00:00', '2016-04-29 10:00:00')

exception_label(98, 1, '2016-05-04 17:00:00', '2016-06-15 11:00:00')

exception_label(172, 1, '2016-03-12 00:00:00', '2016-03-21 09:00:00')

exception_label(190, 1, '2016-01-01 00:00:00', '2016-02-08 09:00:00')
exception_label(190, 1, '2016-12-02 20:00:00', '2016-12-09 11:00:00')

exception_label(191, 1, '2016-04-02 09:00:00', '2016-04-11 04:00:00')

exception_label(231, 1, '2016-03-23 13:00:00', '2016-04-04 09:00:00')

# 235 之前只处理了最后的，现在的处理可以先不处理，先比较，或者想一想处理的时候怎么验证
exception_label(235, 1, '2016-05-18 06:00:00', '2016-06-23 04:00:00')

exception_label(236, 1, '2016-01-24 01:00:00', '2016-03-12 22:00:00')
exception_label(236, 1, '2016-03-16 16:00:00', '2016-03-21 12:00:00')

exception_label(240, 1, '2016-01-01 00:00:00', '2016-01-08 09:00:00')

exception_label(264, 1, '2016-01-01 00:00:00', '2016-02-08 10:00:00')

exception_label(880, 1, '2016-03-18 22:00:00', '2016-05-02 08:00:00')
exception_label(880, 1, '2016-05-02 10:00:00', '2016-05-18 10:00:00')

exception_label(1227, 1, '2016-02-10 19:00:00', '2016-02-26 14:00:00')
exception_label(1227, 1, '2016-04-09 20:00:00', '2016-04-18 19:00:00')

exception_label(1239, 1, '2016-03-19 03:00:00', '2016-03-27 16:00:00')

exception_label(1240, 1, '2016-04-20 23:00:00', '2016-05-02 11:00:00')

exception_label(1246, 1, '2016-01-01 00:00:00', '2016-03-02 19:00:00')

exception_label(1268, 1, '2016-01-19 13:00:00', '2016-01-27 09:00:00')
# exception_label(1268, 1, '2016-02-16 03:00:00', '2016-02-29 15:00:00')
train.loc[(train.building_id == 1268)  & (train.meter == 1) & (train.timestamp >= '2016-02-16 03:00:00') & (train.timestamp <= '2016-02-29 15:00:00'), 'exception'] = 3
test.loc[(test.building_id == 1268)  & (test.meter == 1) & (test.timestamp >= '2017-02-16 03:00:00') & (test.timestamp <= '2017-02-28 23:00:00'), 'exception'] = 3
test.loc[(test.building_id == 1268)  & (test.meter == 1) & (test.timestamp >= '2018-02-16 03:00:00') & (test.timestamp <= '2018-02-28 23:00:00'), 'exception'] = 3

exception_label(1276, 1, '2016-01-01 00:00:00', '2016-01-31 18:00:00')

exception_label(1290, 1, '2016-01-01 00:00:00', '2016-01-31 18:00:00')

exception_label(1303, 1, '2016-01-30 20:00:00', '2016-02-12 02:00:00')

exception_label(1349, 1, '2016-01-05 22:00:00', '2016-01-21 09:00:00')
exception_label(1349, 1, '2016-01-21 17:00:00', '2016-01-27 11:00:00')
exception_label(1349, 1, '2016-01-27 20:00:00', '2016-04-11 11:00:00')
exception_label(1349, 1, '2016-04-11 22:00:00', '2016-04-26 13:00:00')

exception_label(1363, 1, '2016-04-14 09:00:00', '2016-04-21 06:00:00')


exception_label(175, 1, '2016-12-22 19:00:00', '2016-12-31 23:00:00')

exception_label(1307, 1, '2016-12-17 06:00:00', '2016-12-31 23:00:00')

标记bid=43, meter=1异常
标记bid=43, meter=1异常
标记bid=60, meter=1异常
标记bid=60, meter=1异常
标记bid=98, meter=1异常
标记bid=172, meter=1异常
标记bid=190, meter=1异常
标记bid=190, meter=1异常
标记bid=191, meter=1异常
标记bid=231, meter=1异常
标记bid=235, meter=1异常
标记bid=236, meter=1异常
标记bid=236, meter=1异常
标记bid=240, meter=1异常
标记bid=264, meter=1异常
标记bid=880, meter=1异常
标记bid=880, meter=1异常
标记bid=1227, meter=1异常
标记bid=1227, meter=1异常
标记bid=1239, meter=1异常
标记bid=1240, meter=1异常
标记bid=1246, meter=1异常
标记bid=1268, meter=1异常
标记bid=1276, meter=1异常
标记bid=1290, meter=1异常
标记bid=1303, meter=1异常
标记bid=1349, meter=1异常
标记bid=1349, meter=1异常
标记bid=1349, meter=1异常
标记bid=1349, meter=1异常
标记bid=1363, meter=1异常
标记bid=175, meter=1异常
标记bid=1307, meter=1异常


In [19]:
%%time
for bid in train[(train.exception == 3) & (train.meter == 1)].building_id.unique():
    train.loc[(train.building_id == bid) & (train.meter == 1) & (train.exception == 0), 'exception'] = -3
    test.loc[(test.building_id == bid) & (test.meter == 1) & (test.exception != 3), 'exception'] = -3

CPU times: user 11.6 s, sys: 8.91 s, total: 20.5 s
Wall time: 20.5 s


In [20]:
train.exception.value_counts(dropna=False)

 0    16495815
-1     1923669
-3      887630
 3      379772
 1      365029
-2       85465
 2       78720
Name: exception, dtype: int64

In [21]:
test.exception.value_counts(dropna=False)

 0.0    32911252
-1.0     3904132
-3.0     1843538
 NaN     1205584
 1.0      771056
 3.0      766942
-2.0      147834
 2.0      147262
Name: exception, dtype: int64

In [22]:
exception_label(751, 2, '2016-01-01 00:00:00', '2016-02-03 13:00:00')

exception_label(766, 2, '2016-01-01 00:00:00', '2016-02-03 13:00:00')

exception_label(770, 2, '2016-01-08 12:00:00', '2016-02-03 13:00:00')

exception_label(774, 2, '2016-01-01 00:00:00', '2016-02-11 08:00:00')

exception_label(876, 2, '2016-04-07 15:00:00', '2016-05-16 07:00:00')

exception_label(886, 2, '2016-01-01 00:00:00', '2016-01-26 08:00:00')

exception_label(895, 2, '2016-01-12 16:00:00', '2016-04-27 08:00:00')

exception_label(929, 2, '2016-01-22 11:00:00', '2016-02-02 07:00:00')

exception_label(932, 2, '2016-02-04 09:00:00', '2016-04-28 09:00:00')

exception_label(961, 2, '2016-06-17 09:00:00', '2016-07-05 17:00:00')

exception_label(972, 2, '2016-03-01 10:00:00', '2016-06-22 09:00:00')

exception_label(1176, 2, '2016-01-01 00:00:00', '2016-02-10 15:00:00')

exception_label(1189, 2, '2016-01-01 00:00:00', '2016-02-10 15:00:00')

exception_label(1238, 2, '2016-01-20 15:00:00', '2016-05-26 19:00:00')

exception_label(1303, 2, '2016-01-15 12:00:00', '2016-06-06 18:00:00')

标记bid=751, meter=2异常
标记bid=766, meter=2异常
标记bid=770, meter=2异常
标记bid=774, meter=2异常
标记bid=876, meter=2异常
标记bid=886, meter=2异常
标记bid=895, meter=2异常
标记bid=929, meter=2异常
标记bid=932, meter=2异常
标记bid=961, meter=2异常
标记bid=972, meter=2异常
标记bid=1176, meter=2异常
标记bid=1189, meter=2异常
标记bid=1238, meter=2异常
标记bid=1303, meter=2异常


In [23]:
%%time
for bid in train[(train.exception == 3) & (train.meter == 2)].building_id.unique():
    train.loc[(train.building_id == bid) & (train.meter == 2) & (train.exception == 0), 'exception'] = -3
    test.loc[(test.building_id == bid) & (test.meter == 2) & (test.exception != 3), 'exception'] = -3

CPU times: user 7.36 s, sys: 5.48 s, total: 12.8 s
Wall time: 12.8 s


In [24]:
train.exception.value_counts(dropna=False)

 0    16364143
-1     1923669
-3      998165
 3      400909
 1      365029
-2       85465
 2       78720
Name: exception, dtype: int64

In [25]:
test.exception.value_counts(dropna=False)

 0.0    32648628
-1.0     3904132
-3.0     2064168
 NaN     1205408
 3.0      809112
 1.0      771056
-2.0      147834
 2.0      147262
Name: exception, dtype: int64

In [26]:
exception_label(163, 3, '2016-01-31 11:00:00', '2016-12-31 22:00:00')

exception_label(167, 3, '2016-08-01 00:00:00', '2016-11-21 08:00:00')

exception_label(171, 3, '2016-03-18 02:00:00', '2016-12-05 11:00:00')

exception_label(200, 3, '2016-03-22 01:00:00', '2016-12-31 23:00:00')

exception_label(279, 3, '2016-01-01 00:00:00', '2016-04-23 04:00:00')
exception_label(279, 3, '2016-04-25 08:00:00', '2016-12-31 23:00:00')

exception_label(287, 3, '2016-01-01 00:00:00', '2016-06-03 20:00:00')
exception_label(287, 3, '2016-06-04 01:00:00', '2016-08-30 12:00:00')
exception_label(287, 3, '2016-09-01 04:00:00', '2016-12-31 23:00:00')

exception_label(1227, 3, '2016-01-01 00:00:00', '2016-02-26 14:00:00')
exception_label(1227, 3, '2016-04-09 20:00:00', '2016-04-18 19:00:00')

exception_label(1235, 3, '2016-04-04 22:00:00', '2016-04-22 07:00:00')
exception_label(1235, 3, '2016-05-02 18:00:00', '2016-05-09 12:00:00')

exception_label(1240, 3, '2016-04-20 23:00:00', '2016-05-02 11:00:00')

exception_label(1241, 3, '2016-05-22 15:00:00', '2016-12-31 23:00:00')

exception_label(1244, 3, '2016-04-04 22:00:00', '2016-05-06 17:00:00')
exception_label(1244, 3, '2016-08-23 10:00:00', '2016-08-31 19:00:00')

exception_label(1246, 3, '2016-01-01 00:00:00', '2016-03-02 18:00:00')

标记bid=163, meter=3异常
标记bid=167, meter=3异常
标记bid=171, meter=3异常
标记bid=200, meter=3异常
标记bid=279, meter=3异常
标记bid=279, meter=3异常
标记bid=287, meter=3异常
标记bid=287, meter=3异常
标记bid=287, meter=3异常
标记bid=1227, meter=3异常
标记bid=1227, meter=3异常
标记bid=1235, meter=3异常
标记bid=1235, meter=3异常
标记bid=1240, meter=3异常
标记bid=1241, meter=3异常
标记bid=1244, meter=3异常
标记bid=1244, meter=3异常
标记bid=1246, meter=3异常


In [27]:
%%time
for bid in train[(train.exception == 3) & (train.meter == 3)].building_id.unique():
    train.loc[(train.building_id == bid) & (train.meter == 3) & (train.exception == 0), 'exception'] = -3
    test.loc[(test.building_id == bid) & (test.meter == 3) & (test.exception != 3), 'exception'] = -3

CPU times: user 5.79 s, sys: 4.5 s, total: 10.3 s
Wall time: 10.3 s


In [28]:
train.exception.value_counts(dropna=False)

 0    16258788
-1     1923669
-3     1051924
 3      452505
 1      365029
-2       85465
 2       78720
Name: exception, dtype: int64

In [29]:
test.exception.value_counts(dropna=False)

 0.0    32438494
-1.0     3904132
-3.0     2171394
 NaN     1205302
 3.0      912126
 1.0      771056
-2.0      147834
 2.0      147262
Name: exception, dtype: int64

In [30]:
# ## 存疑异常标记

# # 30前期有大量0值
# train.loc[(train.building_id == 30) & (train.meter == 1) & (train.meter_reading < 212), 'exception'] = 2

# # 43大量0值
# train.loc[(train.building_id == 43) & (train.meter == 1) & (train.meter_reading == 0), 'exception'] = 2

# # 28大量0值
# train.loc[(train.building_id == 28) & (train.meter == 1) & (train.meter_reading == 0), 'exception'] = 2

In [31]:
gc.collect()

100

In [33]:
train[['exception']].to_pickle('../output/ashrae-eda-exception-label5/train_exception.pkl')
test[['exception']].to_pickle('../output/ashrae-eda-exception-label5/test_exception.pkl')